In [6]:
import os
from secrets import base_path

paths_dict =  {
    "zenodo_files_results": os.path.join(base_path, "DeepDiveBugReportsWithLogs", "data", "zenodo_files_results.json"),
    "gzoltar_execution_path": os.path.join(base_path, "gzoltar_files", ),
    "defects4j_path": os.path.join(base_path, "gzoltar_files", "defects4j"),
    "gzoltar_path": os.path.join(base_path, "gzoltar_files", "gzoltar"),
    "bugs_data": os.path.join(base_path, "DeepDiveBugReportsWithLogs", "data", "bug_reports_with_stack_traces_details.json"),
    "current_path": os.path.join(base_path, "DeepDiveBugReportsWithLogs", "code", "auxiliary-scripts"),
}

files_to_copy_per_project = {
    "Cli": ".defects4j.config defects4j.build.properties build.xml pom.xml maven.xml"
}

java_version_for_defects4j_execution = "1.8.0_292" # Change according to necessary

In [7]:
%run ../utils.py

import subprocess

zenodo_files_results = json_file_to_dict(paths_dict["zenodo_files_results"])
bugs_data = json_file_to_dict(paths_dict["bugs_data"])
for bug_report in zenodo_files_results["bugs_not_found"]:
    print("************")
    print(bug_report)

    project = bug_report.split("_")[0]
    bug_id = bug_report.split("_")[1]

    bug_report_commit_hash = bugs_data[project][bug_id]["bug_report_commit_hash"]

    defects4j_buggy_folder = os.path.join(paths_dict["gzoltar_execution_path"], bug_report + "_buggy")
    defects4j_copy_folder = os.path.join(paths_dict["gzoltar_execution_path"], bug_report + "_copy")

    # Configuring the environment variables
    # 1. Java
    java_home = subprocess.check_output(["/usr/libexec/java_home", "-v", java_version_for_defects4j_execution]).decode().strip() # Making sure the current Java version is 1.8 (defects4j requirement)
    os.environ['JAVA_HOME'] = java_home
    # 2. defects4j
    os.environ["PATH"] += os.pathsep + paths_dict['defects4j_path'] + "/framework/bin"
    os.environ["D4J_HOME"] = paths_dict['defects4j_path']
    os.environ["TZ"] = 'America/Los_Angeles'
    os.environ["LC_ALL"] = "en_US.UTF-8"
    os.environ["LANG"] = "en_US.UTF-8"
    os.environ["LANGUAGE"] = "en_US.UTF-8"
    # 3. gzoltar
    os.environ['GZOLTAR_AGENT_JAR'] = paths_dict['gzoltar_path'] + "/gzoltar/com.gzoltar.agent.rt/target/com.gzoltar.agent.rt-1.7.3-SNAPSHOT-all.jar"
    os.environ['GZOLTAR_CLI_JAR'] = paths_dict['gzoltar_path'] + "/gzoltar/com.gzoltar.cli/target/com.gzoltar.cli-1.7.3-SNAPSHOT-jar-with-dependencies.jar"

    # ----- defects4j part -----

    # Running defects4j checkout
    defects4j_checkout_cmd = f"defects4j checkout -p {project} -v {bug_id}b -w {defects4j_buggy_folder}"
    print("> " + defects4j_checkout_cmd)
    subprocess.run(defects4j_checkout_cmd, shell=True, check=True)

    # Creating a copy of the folder
    copy_folder_cmd = f"cp -r {defects4j_buggy_folder} {defects4j_copy_folder}"
    subprocess.run(copy_folder_cmd, shell=True, check=True)

    # Checkout in the bug report commit in the copy folder
    checkout_commit_cmd = f"git checkout {bug_report_commit_hash}"
    subprocess.run(checkout_commit_cmd, shell=True, check=True, cwd=defects4j_copy_folder)

    # Copying defects4j important files to the copy folder
    copy_files_cmd = f"cp {files_to_copy_per_project[project]} {defects4j_copy_folder}"
    subprocess.run(copy_files_cmd, shell=True, check=True, cwd=defects4j_buggy_folder)
    cmd = "echo 'org.apache.commons.**.*' > all-classes.txt"
    subprocess.run(cmd, shell=True, check=True, cwd=defects4j_copy_folder)

    if bug_report == "Cli_14":
        remove_non_utf8_characters(defects4j_copy_folder + "/src/test/org/apache/commons/cli2/bug/BugLoopingOptionLookAlikeTest.java")

    # Defects4j compile
    defects4j_compile_cmd = "defects4j compile"
    print("> " + defects4j_compile_cmd)
    subprocess.run(defects4j_compile_cmd, shell=True, check=True, cwd=defects4j_copy_folder)

    # ---- Gzoltar part ------

    # Navigate to the directory
    os.chdir(defects4j_copy_folder)

    # Get test classpath
    test_classpath = subprocess.check_output(["defects4j", "export", "-p", "cp.test"]).decode().strip()
    # Defects4j test
    defects4j_test_cmd = "defects4j test"
    print("> " + defects4j_test_cmd)
    subprocess.run(defects4j_test_cmd, shell=True, check=True, cwd=defects4j_copy_folder)

    # Get source classes directory
    src_classes_dir_output = subprocess.check_output(["defects4j", "export", "-p", "dir.bin.classes"]).decode().strip()
    src_classes_dir = os.path.join(defects4j_copy_folder, src_classes_dir_output)

    # Get test classes directory
    test_classes_dir_output = subprocess.check_output(["defects4j", "export", "-p", "dir.bin.tests"]).decode().strip()
    test_classes_dir = os.path.join(defects4j_copy_folder, test_classes_dir_output)

    print(f"{project}_{bug_id}_copy's classpath: {test_classpath}")
    print(f"{project}_{bug_id}_copy's bin dir: {src_classes_dir}")
    print(f"{project}_{bug_id}_copy's test bin dir: {test_classes_dir}")

    # -------

    # Remove the created folders to free the space
    delete_folders_cmd = f"""
        rm -rf {defects4j_buggy_folder};
        rm -rf {defects4j_copy_folder}"""
    subprocess.run(delete_folders_cmd, shell=True, check=True)

    # Coming back to the project's folder
    os.chdir(paths_dict["current_path"])

    print("Done")
    break

************
Cli_14
> defects4j checkout -p Cli -v 14b -w /Users/lorenapacheco/Concordia/Masters/gzoltar_files/Cli_14_buggy


Checking out c338c8ac to /Users/lorenapacheco/Concordia/Masters/gzoltar_fil OK
Init local repository...................................................... OK
Tag post-fix revision...................................................... OK
Run post-checkout hook..................................................... OK
Excluding broken/flaky tests............................................... OK
Excluding broken/flaky tests............................................... OK
Initialize fixed program version........................................... OK
Apply patch................................................................ OK
Initialize buggy program version........................................... OK
Diff c338c8ac:b7b2069f..................................................... OK
Apply patch................................................................ OK
Tag pre-fix revision....................................................... OK
Check out program version: Cli-14b..................

> defects4j compile


OK
Running ant (compile.tests)................................................ OK
OpenJDK 64-Bit Server VM warning: ignoring option MaxPermSize=1G; support was removed in 8.0
Running ant (export.cp.test)............................................... OK

Running ant (compile.tests)................................................ 

> defects4j test


OK
Running ant (run.dev.tests)................................................ OK
OpenJDK 64-Bit Server VM warning: ignoring option MaxPermSize=1G; support was removed in 8.0


Failing tests: 1
  - org.apache.commons.cli2.bug.Bug27575Test::testRequiredOptions


Running ant (export.dir.bin.classes)....................................... OK

OpenJDK 64-Bit Server VM warning: ignoring option MaxPermSize=1G; support was removed in 8.0
Running ant (export.dir.bin.tests)......................................... 

Cli_14_copy's classpath: /Users/lorenapacheco/.maven/repository/commons-lang/jars/commons-lang-2.1.jar:/Users/lorenapacheco/.maven/repository/junit/jars/junit-3.8.1.jar:/Users/lorenapacheco/.maven/repository/jdepend/jars/jdepend-2.5.jar:/Users/lorenapacheco/Concordia/Masters/gzoltar_files/Cli_14_copy/target/test-classes:/Users/lorenapacheco/Concordia/Masters/gzoltar_files/Cli_14_copy/target/classes:/Users/lorenapacheco/Concordia/Masters/gzoltar_files/Cli_14_copy/file:/Users/lorenapacheco/Concordia/Masters/gzoltar_files/defects4j/framework/projects/lib/junit-4.11.jar:/Users/lorenapacheco/Concordia/Masters/gzoltar_files/defects4j/framework/projects/Cli/lib/commons-lang/commons-lang/2.1/commons-lang-2.1.jar:/Users/lorenapacheco/Concordia/Masters/gzoltar_files/defects4j/framework/projects/Cli/lib/jdepend/jdepend/2.5/jdepend-2.5.jar:/Users/lorenapacheco/Concordia/Masters/gzoltar_files/defects4j/framework/projects/Cli/lib/junit-addons/junit-addons/1.4/junit-addons-1.4.jar:/Users/lorenapachec

OK

